### Установка необходимых пакетов
Читаем из файла `requirements.txt`

**Библиотеки:**
* PyTorch v2.1.0
* Sentencepiece v0.1.99
* Transformers v4.34.1
* Pandas  v2.1.1
* NumPy v1.26.1
* Scikit Learn v1.3.2


In [ ]:
%pip install -r requirements.txt

### Считываем датасет

Файлы берём из папки `/dataset`

Для обучения - `train.csv`

Для валидации - `validate.csv`

Для тестирования - `test.csv`

In [ ]:
import pandas as pd # Импортируем библиотеку Pandas

# Отключаем предупреждения
no_deprecation_warning=True

In [ ]:
train_data = pd.read_csv('./dataset/train.csv')     # Считываем данные для обучения
valid_data = pd.read_csv('./dataset/validate.csv')  # Считываем данные для валидации

### Инициализируем библиотеку BERT (`rubert-tiny`)

Импортируем необходимые модули

In [ ]:
from bert_classifier import BertClassifier  # Импортируем основной класс BertClassifier

Инициализируем `BertClassifier` на `41` группу и `60` эпох

In [ ]:
TORCH_USE_CUDA_DSA=True

classifier = BertClassifier(
    model_path='cointegrated/rubert-tiny',          # Путь до репозитория с дообучаемой моделью
    tokenizer_path='cointegrated/rubert-tiny',      # Путь до токенизатора
    n_classes=41,                                   # Количество классов для обучения
    epochs=60,                                      # Количество эпох обучения
    max_len=512,                                   # Максимальный размер текста
    model_save_path='./output/model.pt'             # Путь до сохранения модели
)

### Подготавливаем данные для обучения и валидации

In [ ]:
classifier.preparation(
    X_train=list(train_data['groups']),             # Обучающие поля таблицы с текстом
    y_train=list(train_data['code']),               # Обучающие поля таблицы с реальными кодами групп
    X_valid=list(valid_data['groups']),             # Валидирующие поля таблицы с текстом
    y_valid=list(valid_data['code'])                # Валидирующие поля таблицы с реальными кодами групп
)

Проверяем возможность обучения нейросети на CUDA ядрах и выводим дополнительную информацию

In [ ]:
import torch
print(f"CUDA поддерживается этой системой: {torch.cuda.is_available()}")
print(f"Версия CUDA: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID текущего CUDA устройства: {torch.cuda.current_device()}")
       
print(f"Имя текущего CUDA устройства: {torch.cuda.get_device_name(cuda_id)}")

### Обучаем нейросеть

In [ ]:
classifier.train()

### Тестируем работу обученной модели

In [ ]:
import time

test_data  = pd.read_csv('./dataset/test.csv')

texts = list(test_data['groups'])
labels = list(test_data['code'])

predictions = [classifier.predict(t) for t in texts]

report = []     # form report variable

for t, l, p in zip(texts, labels, predictions):
    report.append({
        "real_group": l,
        "predicted_group": p['prediction'],
        "isEqual": ("no", "yes")[l == p['prediction']],
        "groups_set": t
    })

# Экспортируем результаты в CSV файл, поддерживаемый Excel
report_df = pd.DataFrame(report)
report_df.to_csv(f"./reports/report{time.time()}.csv", index_label="id", sep=";", encoding="cp1251")

print(f"Точность предсказаний: {(report_df[(report_df['isEqual']=='yes')].count()['isEqual']/report_df.__len__())}")

Выводим таблицу с результатами для сверки

In [ ]:
report_df.head()

### Получаем метрики

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

predictions_all = [p['prediction'] for p in predictions]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions_all, average='macro', zero_division=0)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')